In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def roi(image, vertices):
    mask = np.zeros_like(image)
    mask_color = 255
    cv2.fillPoly(mask, vertices, mask_color)
    masked_img = cv2.bitwise_and(image, mask)
    return masked_img

In [3]:
def draw_lines(lines, image):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

In [8]:
def process(img):
    try:
        h, w, _ = img.shape
        roi_vertices = [
            (200, h),
            (w/2, 2*h/3),
            (w-100, h) 
        ]
        
        
        grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        kernel = np.ones((3,3), np.uint8)
        grayimg = cv2.dilate(grayimg, kernel=kernel)
        
        canny = cv2.Canny(grayimg, 60, 255)
        
        roi_image = roi(canny, np.array([roi_vertices], np.uint32))
        
        hough_lines = cv2.HoughLinesP(roi_image, 1, np.pi/180, 40, minLineLength=10, maxLineGap=5)
        fimg = draw_lines(hough_lines, img)
        return fimg
    
    except Exception:
        return img
    

In [9]:
cap = cv2.VideoCapture("D:/CodeClause_AI_Internship/ROAD_LANE_DETECTION/New folder/Data/Manhattan_Trim.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"XVID")
saved_frame = cv2.VideoWriter("Manhattan_detection.avi", fourcc, 30.0, (frame_width, frame_height))


while cap.isOpened():
    _, frame = cap.read()
    
    try:
        
        frame = process(frame)
        saved_frame.write(frame)
        cv2.imshow("final", frame)
        
        if cv2.waitKey(1) & 0xFF == 27:
            break
            
    except Exception:
        break
        
cap.release()
saved_frame.release()
cv2.destroyAllWindows()